Pythonには、日付のdate、日付時間のdatetime、時間差のtimedeltaの型がある。

なぜか。日付は（普通の数字とは違うが）順序もあるし、足し算、引き算ができるし、（しかし掛け算の意味はわからない）、（数字と違う）独自の表示方法がある。だから、floatとかintとは違う「数値型」として、準備してある。

In [2]:
from datetime import date
t = date.fromisoformat("2021-04-05")

独自の表示は以下のように確かめられる。

In [3]:
print(t)

2021-04-05


日付時間の場合

In [4]:
from datetime import datetime
d = datetime(2021,3,5,12,5,20)

In [5]:
print(d)

2021-03-05 12:05:20


時間の引き算ができる。結果はtimedelta型になる。

In [8]:
d1 = datetime(2021,1,1,0,0,0)
d2 = datetime(2021,4,5,12,5,20)
delta1 = d2 - d1
print( type(delta1) )
print(delta1)

<class 'datetime.timedelta'>
94 days, 12:05:20


各データ型には様々な属性やメソッドがある。

In [10]:
print( t.year )
print( t.month )
print( t.weekday() ) # 月曜日が0、日曜日が6になる

2021
4
0


#### スクレイピングデータを解釈する例

日付を表す文字列は、'3/15(月) 15:49'という、極めて日本的なフォーマットである。

In [22]:
str_date = '3/15(月) 15:49'

datetime.strptime("2021/" + str_date, "%Y/%m/%d(%a) %H:%M")

ValueError: time data '2021/3/15(月) 15:49' does not match format '%Y/%m/%d(%a) %H:%M'

Pythonの標準的な文字列変換では、(月)が処理できないようだ。

ならば、削除するしかない。

それには、正規表現を使い、(月)を空文字で置換する。

In [27]:
str_date = '3/15(月) 15:49'

import re

re.sub('\([月火水木金土日]\)', "", str_date)

'3/15 15:49'

これで、丸括弧の曜日を削除することができた。これを関数にする。

In [28]:
import re
def dateString2Datetime(_string):
    cleaned_string = re.sub('\([月火水木金土日]\)', "", _string)
    return datetime.strptime("2021/" + cleaned_string, "%Y/%m/%d %H:%M")

# test
dateString2Datetime('3/15(月) 15:49')

datetime.datetime(2021, 3, 15, 15, 49)

In [29]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
import time   # タイマーを使うため

#ヤフー出稿メディア別のページ一覧を読み込み
df = pd.read_csv('yahoo_rss_all_list_2021_04_01.csv')

#保存先ファイルを開き、列を作成
with open("***.csv", mode="a", encoding="utf_8_sig", newline='') as f:
  writer = csv.DictWriter(f, ['Media', 'Title', 'Pubdate', 'Link'])
  writer.writeheader()
  
  #スクレイピングの設定
  for index, row in df.iterrows():
    url = row['url']
    # エチケットとして５秒待つ
    time.sleep(5)

    html = requests.get( url )
    soup = BeautifulSoup(html.text, 'html.parser' )

    for news in soup.find_all(class_='newsFeed_item'):
      d = {}
      d["Media"]  = row['media_online']
      d["Title"] = news.select_one('.newsFeed_item_title').text
      # ここを関数で処理
      d["Pubdate"] = dateString2Datetime( news.select_one('.newsFeed_item_date').text )
      d["Link"] = news.a['href']
      print(d)
      writer.writerow(d)


{'Media': 'Yahoo!ニュース_オリジナル_RED_Chair', 'Title': '僕、このまま終わるんじゃないですか――はじめしゃちょーの「ガチ相談」、佐藤二朗の回答は', 'Pubdate': datetime.datetime(2021, 4, 4, 16, 21), 'Link': 'https://news.yahoo.co.jp/articles/0dff7866d65be200fd7430c87688ed17b4f41165'}
{'Media': 'Yahoo!ニュース_オリジナル_RED_Chair', 'Title': '「ロールモデルがいない」三浦瑠麗と蜷川実花、それでも捉われた「昭和の育児」の呪縛', 'Pubdate': datetime.datetime(2021, 3, 15, 15, 49), 'Link': 'https://news.yahoo.co.jp/articles/2fb15e5c12df914138044832fa56fb90903b8a89'}
{'Media': 'Yahoo!ニュース_オリジナル_RED_Chair', 'Title': '寄り添えるか、優しくできるか――綾野剛と常田大希が語るクリエイティブの「原点」', 'Pubdate': datetime.datetime(2021, 3, 12, 12, 14), 'Link': 'https://news.yahoo.co.jp/articles/9c84797c769077f7ea837b60c0566996154bc393'}
{'Media': 'Yahoo!ニュース_オリジナル_RED_Chair', 'Title': '「日本人じゃない」と突きつけられた過去――水原希子、自分らしさとの葛藤の先に', 'Pubdate': datetime.datetime(2021, 3, 11, 12, 5), 'Link': 'https://news.yahoo.co.jp/articles/acce2e42c7213e16cbae53e4b0f98e8855d658b4'}
{'Media': 'Yahoo!ニュース_オリジナル_RED_Chair', 'Title': '「お金は社会に還元して死ぬ」――「暴走族」安藤忠雄79歳、規格外

KeyboardInterrupt: 